In [2]:
import pandas as pd
import joblib
from google.colab import files

df = pd.read_csv('clean_dataset_Romania.csv')

# Notațiile acordate de utilizator pentru fiecare categorie
user_ratings = {
    'nota_personal': 6,
    'nota_facilităţi': 4,
    'nota_curăţenie': 10,
    'nota_confort': 10,
    'nota_raport_calitate/preţ': 10,
    'nota_locaţie': 3,
    'nota_wifi_gratuit': 1
}

# Verificăm dacă suma notelor acordate de utilizator nu depășește numărul maxim de puncte
if sum(user_ratings.values()) > 44:
    print('Suma notelor acordate de utilizator depășește numărul maxim de puncte.')
else:
    # Calculăm scorul de preferință pentru fiecare hotel
    for index, row in df.iterrows():
        preference_score = 0
        for category, user_rating in user_ratings.items():
            preference_score += row[category] * user_rating
        df.loc[index, 'preference_score'] = preference_score

# Lista dvs. de cazări preferate
preferred_accommodations = ['Cabana Sebes Bera', 'Casa de Vacanță S&B', 'Apartament doua camere Retro view']


In [3]:
from sklearn.cluster import KMeans

# Creăm un model KMeans cu un număr specific de clustere
kmeans = KMeans(n_clusters=5, n_init='auto')

# Antrenăm modelul cu datele noastre
kmeans.fit(df[['price', 'nota_personal', 'nota_facilităţi', 'nota_curăţenie', 'nota_confort', 'nota_raport_calitate/preţ', 'nota_locaţie', 'nota_wifi_gratuit']])

# Adăugăm etichetele de cluster la DataFrame
df['cluster'] = kmeans.labels_

# Lista dvs. de cazări preferate
preferred_accommodations = ['Cabana Sebes Bera', 'Casa de Vacanță S&B', 'Apartament doua camere Retro view']

# Identificăm clusterul preferat
preferred_cluster = df[df['name'].isin(preferred_accommodations)]['cluster'].mode()[0]

# Filtrăm DataFrame-ul pentru a include doar hotelurile din clusterul preferat
df = df[df['cluster'] == preferred_cluster]


In [5]:
# Aplicăm filtrele
max_budget = 200
df = df[df['price'] <= max_budget]

preferred_region = 'Braşov'
df = df[df['region'] == preferred_region]

# Identificăm facilitățile care sunt disponibile la cazări preferate
preferred_facilities = df[df['name'].isin(preferred_accommodations)].iloc[:, -37:].sum(axis=0)

# Calculăm scorul de potrivire a facilităților pentru fiecare hotel
for index, row in df.iterrows():
    matching_facilities_score = sum(row[-37:] * preferred_facilities)
    df.loc[index, 'preference_score'] += matching_facilities_score

# Afișăm primele 5 hoteluri cu cel mai mare scor de preferință
print(df.nlargest(5, 'preference_score'))

# Salvarea și Descărcarea Modelului KMeans
# joblib.dump(kmeans, 'recommendation_model.pkl')
# files.download('recommendation_model.pkl')

                                  name  \
339  Apartament doua camere Retro view   
739                              Tudor   
343                         Studio ANA   
713             SECRET boutique Hostel   
783                VivamusApartmentsBV   

                                               address  postal_code  country  \
339                    Strada Negoiu, Făgăraş, România          NaN  România   
739           George Călinescu, 500170 Brașov, România     500170.0  România   
343  Strada Tăbăcari, BL 14, AP 11, 505200 Făgăraş,...     505200.0  România   
713      Strada Alecu Russo 2C, 500020 Brașov, România     500020.0  România   
783       2 Strada Orizontului, 507190 Brașov, România     507190.0  România   

     region   latitude  longitude              check_in             check_out  \
339  Braşov  45.833955  24.974558  De la 14:00 la 20:00  Disponibil 24 de ore   
739  Braşov  45.677021  25.610055  De la 15:00 la 23:30  De la 07:00 la 11:00   
343  Braşov  45.841242 

In [6]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Încărcarea dataset-ului
df = pd.read_csv('clean_dataset_Romania.csv')

# Codificarea one-hot
df = pd.get_dummies(df, columns=['type', 'region', 'persons', 'room_type', 'mic_dejun', 'stars'])

# Creează o listă cu numele coloanelor codificate cu one-hot
one_hot_columns = [col for col in df.columns if 'type_' in col or 'region_' in col or 'persons_'
                   in col or 'room_type_' in col or 'mic_dejun_' in col or 'stars_' in col]

# Selectați caracteristicile pe care doriți să le utilizați (de exemplu, ratingurile)
selected_columns = one_hot_columns + ['price', 'nota_personal', 'nota_facilităţi', 'nota_curăţenie', 'nota_confort', 'nota_raport_calitate/preţ', 'nota_locaţie', 'nota_wifi_gratuit',
                                      'vedere_la_oraș', 'menaj_zilnic', 'canale_prin_satelit', 'zonă_de_luat_masa_în_aer_liber', 'cadă',
                                      'facilităţi_de_călcat', 'izolare_fonică', 'terasă_la_soare', 'pardoseală_de_gresie/marmură',
                                      'papuci_de_casă', 'uscător_de_rufe', 'animale_de_companie', 'încălzire', 'birou', 'mobilier_exterior',
                                      'alarmă_de_fum', 'vedere_la_grădină', 'cuptor', 'cuptor_cu_microunde', 'zonă_de_relaxare', 'canapea',
                                      'intrare_privată', 'fier_de_călcat', 'mașină_de_cafea', 'plită_de_gătit', 'extinctoare', 'cană_fierbător',
                                      'grădină', 'ustensile_de_bucătărie', 'maşină_de_spălat', 'balcon', 'pardoseală_de_lemn_sau_parchet',
                                      'aparat_pentru_prepararea_de_ceai/cafea', 'zonă_de_luat_masa', 'canale_prin_cablu', 'aer_condiţionat',
                                      'masă', 'suport_de_haine', 'cadă_sau_duş', 'frigider']

# Creați o matrice de caracteristici
X = df[selected_columns].values

# Aplicam SVD
svd = TruncatedSVD(n_components=5)
X_reduced = svd.fit_transform(X)

# Calculam similaritățile între hoteluri
similarities = cosine_similarity(X_reduced)

# Găsim hotelurile cele mai similare cu primul hotel
top5_similar = np.argsort(similarities[0])[-6:-1][::-1]

# Afișam numele hotelurilor cele mai similare
print(df.iloc[top5_similar]['name'])


542                      Pensiunea Luiza
1913                    Euro House Hotel
2478    Old Town Relaxing&Cozy Apartment
577                          Vila Condor
1485                      Williams House
Name: name, dtype: object
